In [1]:
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *

import sys

import pandas as pd

df = pd.read_csv("vocabulary.csv")
df['proficiency']=None
titles = df.keys()

def words_index(number):
    word_index = []
    for index in df.sample(n=number).index:
        word_index.append(index)
    return word_index

#Build a function that performs as a quiz and the form of the quiz is mutiple choice.
def quiz(number,df):
    import random
    result = {}

    # Quiz the words whose proficiency is -1 or 0
    unknown_df = df[df[titles[2]]== -1]
    familiar_df = df[df[titles[2]] == 0]

    #Quiz words whose proficiency is -1 first.
    if len(unknown_df) >= number:
        quiz_df = unknown_df
    #Quiz words whose proficientcy is 0 if the number of '-1' words is not big enough.
    elif len(unknown_df)+len(familiar_df) >= number:
        quiz_df = unknown_df.join(familiar_df)
    # Else: Quiz the whole word list
    else:
#         return None
        quiz_df = df            

    for i in range(number):
        r = []
        choices = list(df.sample(n=3, replace = True)[titles[1]])  #choose 3 definition  randomly from the whole word list.
        solution = quiz_df[titles[1]][i]
        index = quiz_df[titles[3]][i]
        # Produce choices in a random order.
        flag = random.randint(0,3)
        choices.insert(flag, solution)
        r.append(index)
        r.append(choices)
        r.append(flag)
        result[quiz_df[titles[0]][i]] = tuple(r)

    return result

class QuizWidget(QWidget):
    #this class creates group of radio buttons from given list of labels
    #constructor
    def __init__(self, quiz_dict, *args, **kwargs):
        super(QWidget, self).__init__(*args, **kwargs)

        self.result_dict = {}
        self.quiz_iter = iter(quiz_dict.items())
        self.i = next(self.quiz_iter)
        self.word,self.choices,self.index, self.answer = self.i[0],self.i[1][1],self.i[1][0], self.i[1][2]
        self.instruction = QLabel('Please select the correct definition for: '+self.word)
        # self.instruction.setAlignment(Qt.AlignCenter)
        self.radio_button_group = QButtonGroup()

        #create radio buttons
        self.radio_button_list=[]
        for each in self.choices:
            self.radio_button_list.append(QRadioButton(each))

        #create layout for radio buttons and add them
        self.radio_button_layout = QVBoxLayout()

        #add buttons to the layout and button group
        counter = 0
        for each in self.radio_button_list:
            self.radio_button_layout.addWidget(each)
            self.radio_button_group.addButton(each)
            self.radio_button_group.setId(each, counter)
            counter += 1

        self.radio_button_group.buttonClicked.connect(self.button_clicked)

        self.mainlayout = QVBoxLayout()
        self.mainlayout.addWidget(self.instruction)
        self.mainlayout.addLayout(self.radio_button_layout)
        self.mainlayout.setAlignment(Qt.AlignCenter)
        self.setLayout(self.mainlayout)

    #method to find out the selected button
    def button_clicked(self):
        s = self.radio_button_group.checkedId()
        if s == self.answer:
            self.result_dict[self.word] ='Correct!'
            if df['proficiency'][self.index]:
                df['proficiency'][self.index]+=1
            else:
                df['proficiency'][self.index]=1
        else:
            self.result_dict[self.word] =f'Wrong! Solution is {self.answer}'
            if df['proficiency'][self.index]:
                df['proficiency'][self.index]-=1
            else:
                df['proficiency'][self.index]=-1  

        self.next_quiz()

    def next_quiz(self):
        try:
            self.i = next(self.quiz_iter)
        except:
            print('done')
            for i in range(4):
                self.radio_button_list[i].hide()
            self.instruction.setText('Great work! Here are your results:')
            self.result_label = QLabel(str(self.result_dict))
            self.mainlayout.addWidget(self.result_label)
            return
        
        self.word,self.choices,self.index, self.answer = self.i[0],self.i[1][1],self.i[1][0], self.i[1][2]
        self.instruction.setText('Please select the correct definition for: '+self.word)
#         self.word,self.choices,self.answer = self.i[0],self.i[1][0],self.i[1][1]
        

        for i in range(4):
            self.radio_button_list[i].setText(self.choices[i])


class MemorizeWidget(QWidget):
    def __init__(self, quiz_num=10, *args, **kwargs):
        super(QWidget, self).__init__(*args, **kwargs)
        
        #layout for memorizing mode
        buttonlayout = QHBoxLayout()
        buttonlist = ["I don't know","Looks familiar","I know this"]
        buttonlayout.setContentsMargins(25,25,25,25) 
        i = -1
        for b in buttonlist:
            button = QPushButton(b)
            button.pressed.connect(lambda i = i: self.menu_btn_pushed(i))
            buttonlayout.addWidget(button)
            i+=1
        
        #layout for display screen (flashcard)
        self.quiz_num = quiz_num
        self.index_list = words_index(self.quiz_num)
        displaylayout = QVBoxLayout()
        displaylayout.setContentsMargins(50,50,75,50)
        self.selected_index = 0
        self.card = QPushButton(df[titles[0]][self.index_list[self.selected_index]])
        self.card.setMinimumWidth(500)
        self.card.setMinimumHeight(250)
        self.card.clicked.connect(lambda x: self.card_clicked(self.card))
        displaylayout.addWidget(self.card)
        displaylayout.addLayout(buttonlayout)
    
        #layout for main window
        mainlayout = QHBoxLayout()
        # mainlayout.addLayout(menulayout)
        mainlayout.addLayout(displaylayout)
        
        self.setLayout(mainlayout)
            
    def menu_btn_pushed(self,i):
        if self.selected_index == self.quiz_num:
            print(df.iloc[self.index_list])
        elif df[titles[2]][self.index_list[self.selected_index]]==None:
            df[titles[2]][self.index_list[self.selected_index]]= i
        else:
            df[titles[2]][self.index_list[self.selected_index]]+= i
        self.selected_index+=1
        self.card_clicked(self.card)

    def card_clicked(self,c):
        if self.selected_index >= self.quiz_num:
            c.setText('You have finished all the words for this session!')
        elif c.text() == df[titles[0]][self.index_list[self.selected_index]]:
            c.setText('Definition:\n'+df[titles[1]][self.index_list[self.selected_index]])
        else:
            c.setText(df[titles[0]][self.index_list[self.selected_index]])

class CustomDialog(QInputDialog):
    def __init__(self, *args, **kwargs):
        super(CustomDialog, self).__init__(*args, **kwargs)

        self.setComboBoxItems([str(i) for i in range(10,60,10)])
        self.setLabelText('How many words would you like to memorize?')
        
#our class: MainWindow; qt parent class: QMainWindow
class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        #must always call the super init function or it won't work
        super(MainWindow, self).__init__(*args,**kwargs)

        #set window title, function of QMainWindow Class
        self.setWindowTitle("WordPass: My Awesome Vocabulary Builder")

        #button layout
        memorize_btn = QPushButton('Memorize')
        memorize_btn.clicked.connect(self.memorize_clicked)
        quiz_btn = QPushButton('Quiz')
        quiz_btn.clicked.connect(self.quiz_clicked)
        button_layout = QHBoxLayout()
        button_layout.addWidget(memorize_btn)
        button_layout.addWidget(quiz_btn)


        initial_layout = QVBoxLayout()
        # initial_layout.setAlignment(Qt.AlignTop)
        initial_layout.setSpacing(100)
        initial_layout.setContentsMargins(150,150,150,150)
        welcome = QLabel("Welcome to WordPass, your awesome vocabulary builder")
        welcome.setAlignment(Qt.AlignCenter)
        font = QFont("Times", 15)
        welcome.setFont(font)
        description = QLabel(" Please select your goal today.")
        description.setAlignment(Qt.AlignCenter)
        initial_layout.addWidget(welcome)
        initial_layout.addWidget(description)
        initial_layout.addLayout(button_layout)

        initial_widget = QWidget()
        initial_widget.setLayout(initial_layout)

        self.setCentralWidget(initial_widget)
        
    def memorize_clicked(self):
        dlg = CustomDialog()
        if dlg.exec_():
            i = int(dlg.textValue())
            print(i)
            self.setCentralWidget(MemorizeWidget(quiz_num = i))
        
    def quiz_clicked(self):
        dlg = CustomDialog()
        if dlg.exec_():
            i = int(dlg.textValue())
        x= quiz(i,df)
        self.setCentralWidget(QuizWidget(x))

#sys.argv allows pass command line arguments to application
app = QApplication(sys.argv)

#create window instance (after create QApplication instantce but before enter event loop)
window = MainWindow() 
window.show()

#enter the event loop

app.exec_()

/Users/becky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/becky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


done


0